# Multi-Source Fundamentals: Sharadar + Custom LSEG Data

This notebook demonstrates how to combine **two fundamental data sources** in a single strategy:
- **Sharadar SF1**: 80+ quarterly metrics via Pipeline
- **Custom LSEG**: Your proprietary fundamental data via CustomFundamentals

**What You'll Learn:**
- Access both Sharadar and custom fundamentals simultaneously
- Compare metrics from different sources
- Build consensus scores from multiple sources
- Handle missing data across sources
- Run a complete backtest mixing both datasets

**Use Cases:**
- **Data Quality Checks**: Compare Sharadar vs LSEG for same metric
- **Coverage Expansion**: Use LSEG where Sharadar is missing, vice versa
- **Consensus Signals**: Only trade when both sources agree
- **Alpha Discovery**: Find discrepancies between sources

## Part 1: Setup and Data Loading

In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Add custom_data to path for imports
sys.path.insert(0, '/app/examples/custom_data')

from zipline.pipeline import Pipeline
from zipline.pipeline.data import EquityPricing
from zipline.pipeline.data.sharadar import SharadarFundamentals
from zipline.pipeline.loaders.sharadar_fundamentals import make_sharadar_fundamentals_loader
from zipline.pipeline.engine import SimplePipelineEngine
from zipline.pipeline.domain import US_EQUITIES
from zipline.pipeline.factors import Returns, SimpleMovingAverage
from zipline.data.bundles import load, register
from zipline.data.bundles.sharadar_bundle import sharadar_bundle

# Import custom fundamentals database class
from database_class_approach import CustomFundamentals

# Set display options
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 25)
pd.set_option('display.width', 1500)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (14, 7)

print("✓ Imports complete")

## Load Both Data Sources

In [ ]:
print("="*80)
print("Loading Multi-Source Fundamentals")
print("="*80)

# Step 1: Register and load Sharadar bundle
print("\nStep 1/4: Loading Sharadar bundle...")
register('sharadar', sharadar_bundle())
bundle_data = load('sharadar')
print("  ✓ Sharadar bundle loaded")

# Step 2: Create Sharadar fundamentals loader
print("\nStep 2/4: Creating Sharadar fundamentals loader...")
sharadar_loader = make_sharadar_fundamentals_loader('sharadar')
print(f"  ✓ Sharadar loader created: {sharadar_loader.fundamentals_path}")

# Step 3: Load custom fundamentals database
print("\nStep 3/4: Loading custom LSEG fundamentals database...")
from zipline.data.custom import CustomSQLiteLoader

custom_fundamentals_path = '/root/.zipline/data/custom/fundamentals_zipline.db'
custom_loader = CustomSQLiteLoader(custom_fundamentals_path)
print(f"  ✓ Custom LSEG loader created: {custom_fundamentals_path}")

# Step 4: Find valid trading session
print("\nStep 4/4: Finding valid trading session...")
trading_calendar = bundle_data.equity_daily_bar_reader.trading_calendar
sessions = trading_calendar.sessions_in_range(
    pd.Timestamp('2024-11-01'),
    pd.Timestamp('2024-11-15')
)
test_date = sessions[-1]
print(f"  ✓ Using test date: {test_date.date()}")

# Create Pipeline engine with multi-source loader
def get_loader(column):
    """Route columns to appropriate loader."""
    if hasattr(column, 'dataset'):
        if column.dataset == SharadarFundamentals:
            return sharadar_loader
        elif column.dataset == CustomFundamentals:
            return custom_loader
    return sharadar_loader  # Default

engine = SimplePipelineEngine(
    get_loader=get_loader,
    asset_finder=bundle_data.asset_finder,
    default_domain=US_EQUITIES,
)
print("\n✓ Multi-source Pipeline engine ready")
print("="*80)

## Part 2: Data Comparison - Sharadar vs Custom LSEG

In [ ]:
# Create pipeline with metrics from BOTH sources
comparison_pipeline = Pipeline(
    columns={
        # Sharadar metrics (SF1)
        'sharadar_revenue': SharadarFundamentals.revenue.latest,
        'sharadar_netinc': SharadarFundamentals.netinc.latest,
        'sharadar_roe': SharadarFundamentals.roe.latest,
        'sharadar_pe': SharadarFundamentals.pe.latest,
        'sharadar_de': SharadarFundamentals.de.latest,
        'sharadar_marketcap': SharadarFundamentals.marketcap.latest,
        
        # Custom LSEG metrics
        'lseg_revenue': CustomFundamentals.Revenue.latest,
        'lseg_netincome': CustomFundamentals.NetIncome.latest,
        'lseg_roe': CustomFundamentals.ROE.latest,
        'lseg_pe': CustomFundamentals.PERatio.latest,
        'lseg_de': CustomFundamentals.DebtToEquity.latest,
        'lseg_assets': CustomFundamentals.TotalAssets.latest,
    },
)

print("Running multi-source comparison pipeline...")
comparison_data = engine.run_pipeline(comparison_pipeline, test_date, test_date)

print(f"\n{'='*80}")
print("Multi-Source Data Comparison")
print(f"{'='*80}")
print(f"Total assets: {len(comparison_data):,}")
print(f"\nData coverage by source:")
print(f"  Sharadar metrics: {comparison_data[['sharadar_revenue', 'sharadar_roe']].dropna(how='all').shape[0]:,} assets")
print(f"  LSEG metrics: {comparison_data[['lseg_revenue', 'lseg_roe']].dropna(how='all').shape[0]:,} assets")

# Find assets with data from BOTH sources
both_sources = comparison_data[
    comparison_data['sharadar_roe'].notna() &
    comparison_data['lseg_roe'].notna()
]
print(f"  Both sources: {len(both_sources):,} assets")

print(f"\nSample comparison (stocks with both Sharadar + LSEG data):\n")
if len(both_sources) > 0:
    sample = both_sources.head(10)
    display_cols = ['sharadar_roe', 'lseg_roe', 'sharadar_pe', 'lseg_pe', 'sharadar_de', 'lseg_de']
    print(sample[display_cols].to_string())
else:
    print("  Note: No stocks have data from both sources on this date.")
    print("  This is expected if your custom database has a limited universe.")
    print("\n  Showing Sharadar data:")
    sharadar_only = comparison_data[comparison_data['sharadar_roe'].notna()].head(10)
    print(sharadar_only[['sharadar_revenue', 'sharadar_roe', 'sharadar_pe']].to_string())

## Part 3: Data Quality Check - Compare ROE from Both Sources

For stocks where both sources have data, compare the ROE values to check consistency.

In [ ]:
if len(both_sources) > 5:
    # Calculate difference between sources
    both_sources_copy = both_sources.copy()
    both_sources_copy['roe_diff'] = abs(both_sources_copy['sharadar_roe'] - both_sources_copy['lseg_roe'])
    both_sources_copy['roe_pct_diff'] = (
        both_sources_copy['roe_diff'] / both_sources_copy['sharadar_roe'].abs() * 100
    )
    
    print("ROE Comparison Statistics:")
    print(f"  Mean difference: {both_sources_copy['roe_diff'].mean():.4f}")
    print(f"  Median difference: {both_sources_copy['roe_diff'].median():.4f}")
    print(f"  Max difference: {both_sources_copy['roe_diff'].max():.4f}")
    print(f"\n  Correlation: {both_sources['sharadar_roe'].corr(both_sources['lseg_roe']):.4f}")
    
    # Plot comparison
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    plt.scatter(both_sources['sharadar_roe'] * 100, both_sources['lseg_roe'] * 100, alpha=0.6)
    plt.xlabel('Sharadar ROE (%)', fontsize=11)
    plt.ylabel('LSEG ROE (%)', fontsize=11)
    plt.title('ROE Comparison: Sharadar vs LSEG', fontsize=12, fontweight='bold')
    plt.plot([-50, 50], [-50, 50], 'r--', alpha=0.5, label='Perfect Agreement')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    both_sources_copy['roe_pct_diff'].hist(bins=20)
    plt.xlabel('ROE % Difference', fontsize=11)
    plt.ylabel('Frequency', fontsize=11)
    plt.title('Distribution of ROE Differences', fontsize=12, fontweight='bold')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("Not enough overlapping data for comparison plot.")
    print("Your custom LSEG database may have a limited stock universe.")

## Part 4: Multi-Source Screening Strategy

Build a screening strategy that combines insights from both sources:
1. **Primary Source**: Use Sharadar (broader coverage)
2. **Validation**: Use LSEG where available to confirm signals
3. **Consensus**: Higher confidence when both sources agree

In [ ]:
# Strategy: Quality stocks with consensus from both sources (where available)
strategy_pipeline = Pipeline(
    columns={
        # Sharadar fundamentals (primary)
        's_revenue': SharadarFundamentals.revenue.latest,
        's_netinc': SharadarFundamentals.netinc.latest,
        's_roe': SharadarFundamentals.roe.latest,
        's_pe': SharadarFundamentals.pe.latest,
        's_de': SharadarFundamentals.de.latest,
        's_marketcap': SharadarFundamentals.marketcap.latest,
        
        # LSEG fundamentals (validation)
        'l_roe': CustomFundamentals.ROE.latest,
        'l_pe': CustomFundamentals.PERatio.latest,
        'l_de': CustomFundamentals.DebtToEquity.latest,
    },
)

print("Running multi-source strategy pipeline...")
strategy_data = engine.run_pipeline(strategy_pipeline, test_date, test_date)

# Filter for quality stocks using Sharadar (primary criteria)
quality_sharadar = strategy_data[
    (strategy_data['s_roe'] > 0.15) &       # ROE > 15%
    (strategy_data['s_pe'] > 0) &
    (strategy_data['s_pe'] < 25) &         # P/E < 25
    (strategy_data['s_de'] < 2) &          # D/E < 2
    (strategy_data['s_netinc'] > 0) &      # Profitable
    (strategy_data['s_marketcap'] > 1e9)   # Market cap > $1B
].copy()

print(f"\n{'='*80}")
print("Multi-Source Quality Screen Results")
print(f"{'='*80}")
print(f"\nQuality stocks (Sharadar criteria): {len(quality_sharadar)}")

# Add consensus scoring
quality_sharadar['has_lseg_data'] = quality_sharadar['l_roe'].notna()
quality_sharadar['consensus_score'] = 0

# Add points for Sharadar quality
quality_sharadar.loc[quality_sharadar['s_roe'] > 0.20, 'consensus_score'] += 1
quality_sharadar.loc[quality_sharadar['s_pe'] < 20, 'consensus_score'] += 1
quality_sharadar.loc[quality_sharadar['s_de'] < 1, 'consensus_score'] += 1

# Add bonus points where LSEG confirms
lseg_confirms_roe = (
    quality_sharadar['l_roe'].notna() &
    (quality_sharadar['l_roe'] > 0.15)
)
quality_sharadar.loc[lseg_confirms_roe, 'consensus_score'] += 2  # Bonus for LSEG confirmation

# Sort by consensus score
quality_sharadar = quality_sharadar.sort_values('consensus_score', ascending=False)

print(f"\nTop 20 Quality Stocks (by consensus score):\n")
top_picks = quality_sharadar.head(20)
display_cols = ['s_roe', 'l_roe', 's_pe', 'l_pe', 's_de', 'has_lseg_data', 'consensus_score']
print(top_picks[display_cols].to_string())

# Show breakdown
print(f"\nConsensus Score Breakdown:")
print(quality_sharadar.groupby('consensus_score').size().sort_index(ascending=False).to_string())

# High conviction picks (both sources agree)
high_conviction = quality_sharadar[
    quality_sharadar['has_lseg_data'] &
    (quality_sharadar['consensus_score'] >= 4)
]
print(f"\n✓ High conviction picks (LSEG confirms + high score): {len(high_conviction)} stocks")

## Part 5: Coverage Analysis

Understand where each data source provides coverage and where they overlap.

In [ ]:
# Coverage analysis
coverage_analysis = pd.DataFrame({
    'Metric': ['Revenue', 'ROE', 'P/E', 'D/E'],
    'Sharadar Coverage': [
        comparison_data['sharadar_revenue'].notna().sum(),
        comparison_data['sharadar_roe'].notna().sum(),
        comparison_data['sharadar_pe'].notna().sum(),
        comparison_data['sharadar_de'].notna().sum(),
    ],
    'LSEG Coverage': [
        comparison_data['lseg_revenue'].notna().sum(),
        comparison_data['lseg_roe'].notna().sum(),
        comparison_data['lseg_pe'].notna().sum(),
        comparison_data['lseg_de'].notna().sum(),
    ],
})

# Calculate overlap
coverage_analysis['Both Sources'] = [
    (comparison_data['sharadar_revenue'].notna() & comparison_data['lseg_revenue'].notna()).sum(),
    (comparison_data['sharadar_roe'].notna() & comparison_data['lseg_roe'].notna()).sum(),
    (comparison_data['sharadar_pe'].notna() & comparison_data['lseg_pe'].notna()).sum(),
    (comparison_data['sharadar_de'].notna() & comparison_data['lseg_de'].notna()).sum(),
]

coverage_analysis['Sharadar %'] = (
    coverage_analysis['Sharadar Coverage'] / len(comparison_data) * 100
).round(1)

coverage_analysis['LSEG %'] = (
    coverage_analysis['LSEG Coverage'] / len(comparison_data) * 100
).round(1)

print("\n" + "="*80)
print("Coverage Analysis")
print("="*80)
print(coverage_analysis.to_string(index=False))

# Visualization
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(coverage_analysis))
width = 0.25

ax.bar(x - width, coverage_analysis['Sharadar Coverage'], width, label='Sharadar', alpha=0.8)
ax.bar(x, coverage_analysis['LSEG Coverage'], width, label='LSEG', alpha=0.8)
ax.bar(x + width, coverage_analysis['Both Sources'], width, label='Both', alpha=0.8)

ax.set_xlabel('Metric', fontsize=12)
ax.set_ylabel('Number of Assets', fontsize=12)
ax.set_title('Data Coverage: Sharadar vs LSEG', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(coverage_analysis['Metric'])
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## Part 6: Runnable Backtest Strategy

Now let's run a complete backtest using both data sources!

In [ ]:
from zipline import run_algorithm
from zipline.api import (
    attach_pipeline,
    pipeline_output,
    order_target_percent,
    record,
    schedule_function,
    date_rules,
    time_rules,
)

# ============================================================================
# STRATEGY CONFIGURATION
# ============================================================================

TOP_N_STOCKS = 15
REBALANCE_FREQ = 'monthly'

# ============================================================================
# PIPELINE FACTORY
# ============================================================================

def make_multi_source_pipeline():
    """
    Create pipeline combining Sharadar + LSEG fundamentals.
    
    Strategy:
    - Use Sharadar as primary source (broader coverage)
    - Use LSEG for validation where available
    - Prefer stocks where both sources show quality metrics
    """
    # Sharadar metrics
    s_roe = SharadarFundamentals.roe.latest
    s_pe = SharadarFundamentals.pe.latest
    s_de = SharadarFundamentals.de.latest
    s_netinc = SharadarFundamentals.netinc.latest
    s_marketcap = SharadarFundamentals.marketcap.latest
    
    # LSEG metrics (for confirmation)
    l_roe = CustomFundamentals.ROE.latest
    l_pe = CustomFundamentals.PERatio.latest
    
    # Quality screen (Sharadar)
    quality_screen = (
        (s_roe > 0.15) &      # ROE > 15%
        (s_pe > 0) &
        (s_pe < 25) &         # P/E < 25
        (s_de < 2) &          # D/E < 2
        (s_netinc > 0) &      # Profitable
        (s_marketcap > 1e9)   # $1B+ market cap
    )
    
    return Pipeline(
        columns={
            's_roe': s_roe,
            's_pe': s_pe,
            's_de': s_de,
            's_marketcap': s_marketcap,
            'l_roe': l_roe,
            'l_pe': l_pe,
        },
        screen=quality_screen,
    )

# ============================================================================
# STRATEGY LOGIC
# ============================================================================

def initialize(context):
    """Initialize strategy."""
    # Attach multi-source pipeline
    attach_pipeline(make_multi_source_pipeline(), 'multi_source')
    
    # Schedule rebalancing
    if REBALANCE_FREQ == 'monthly':
        schedule_function(
            rebalance,
            date_rules.month_start(),
            time_rules.market_open(hours=1)
        )
    else:
        schedule_function(
            rebalance,
            date_rules.week_start(),
            time_rules.market_open(hours=1)
        )
    
    # Daily recording
    schedule_function(
        record_vars,
        date_rules.every_day(),
        time_rules.market_close()
    )
    
    context.stocks_held = []
    print(f"\n{'='*80}")
    print(f"Multi-Source Fundamentals Strategy Initialized")
    print(f"{'='*80}")
    print(f"Top N stocks: {TOP_N_STOCKS}")
    print(f"Rebalance frequency: {REBALANCE_FREQ}")
    print(f"Data sources: Sharadar SF1 + Custom LSEG")
    print(f"{'='*80}\n")


def before_trading_start(context, data):
    """Update pipeline data daily."""
    context.pipeline_data = pipeline_output('multi_source')


def rebalance(context, data):
    """Monthly/weekly rebalancing."""
    pipeline_data = context.pipeline_data
    
    if len(pipeline_data) == 0:
        print(f"[{context.datetime.date()}] No stocks passed quality screen")
        # Exit all positions
        for stock in context.portfolio.positions:
            order_target_percent(stock, 0)
        context.stocks_held = []
        return
    
    # Calculate consensus score
    pipeline_data = pipeline_data.copy()
    pipeline_data['consensus_score'] = 0
    
    # Points for Sharadar quality
    pipeline_data.loc[pipeline_data['s_roe'] > 0.20, 'consensus_score'] += 2
    pipeline_data.loc[pipeline_data['s_pe'] < 20, 'consensus_score'] += 1
    pipeline_data.loc[pipeline_data['s_de'] < 1, 'consensus_score'] += 1
    
    # Bonus where LSEG confirms
    lseg_confirms = (
        pipeline_data['l_roe'].notna() &
        (pipeline_data['l_roe'] > 0.15)
    )
    pipeline_data.loc[lseg_confirms, 'consensus_score'] += 2
    
    # Select top N by consensus score, then by ROE
    ranked = pipeline_data.sort_values(
        ['consensus_score', 's_roe'],
        ascending=[False, False]
    )
    
    target_stocks = ranked.head(TOP_N_STOCKS).index.tolist()
    
    # Equal weight portfolio
    target_weight = 1.0 / len(target_stocks) if target_stocks else 0
    
    # Rebalance
    for stock in target_stocks:
        if data.can_trade(stock):
            order_target_percent(stock, target_weight)
    
    # Exit positions not in target
    for stock in context.portfolio.positions:
        if stock not in target_stocks and data.can_trade(stock):
            order_target_percent(stock, 0)
    
    context.stocks_held = target_stocks
    
    # Log rebalancing
    lseg_confirmed = ranked.head(TOP_N_STOCKS)['l_roe'].notna().sum()
    print(f"[{context.datetime.date()}] Rebalanced: {len(target_stocks)} stocks, "
          f"{lseg_confirmed} confirmed by LSEG")


def record_vars(context, data):
    """Record daily metrics."""
    record(
        num_positions=len(context.portfolio.positions),
        portfolio_value=context.portfolio.portfolio_value,
    )


def analyze(context, perf):
    """Analyze backtest results."""
    print(f"\n{'='*80}")
    print("Backtest Complete - Performance Summary")
    print(f"{'='*80}")
    
    # Calculate metrics
    returns = perf['returns']
    total_return = (perf['portfolio_value'].iloc[-1] / perf['portfolio_value'].iloc[0] - 1) * 100
    
    print(f"\nTotal Return: {total_return:.2f}%")
    print(f"Start Value: ${perf['portfolio_value'].iloc[0]:,.2f}")
    print(f"End Value: ${perf['portfolio_value'].iloc[-1]:,.2f}")
    print(f"\nAvg Daily Return: {returns.mean() * 100:.3f}%")
    print(f"Volatility (daily): {returns.std() * 100:.3f}%")
    print(f"Sharpe Ratio: {returns.mean() / returns.std() * np.sqrt(252):.3f}" if returns.std() > 0 else "N/A")
    print(f"\nMax Positions: {perf['num_positions'].max():.0f}")
    print(f"Avg Positions: {perf['num_positions'].mean():.1f}")
    
    print(f"\n{'='*80}\n")
    
    return perf

print("✓ Strategy functions defined")
print("  Ready to run backtest!")

## Run the Backtest

In [ ]:
# Backtest parameters
START_DATE = pd.Timestamp('2023-01-01', tz='UTC')
END_DATE = pd.Timestamp('2024-11-01', tz='UTC')
CAPITAL_BASE = 100000  # $100K starting capital

print(f"Running backtest from {START_DATE.date()} to {END_DATE.date()}...\n")

try:
    results = run_algorithm(
        start=START_DATE,
        end=END_DATE,
        initialize=initialize,
        before_trading_start=before_trading_start,
        analyze=analyze,
        capital_base=CAPITAL_BASE,
        bundle='sharadar',
    )
    
    print("\n✓ Backtest completed successfully!")
    
except Exception as e:
    print(f"\n❌ Backtest failed: {e}")
    import traceback
    traceback.print_exc()
    results = None

## Visualize Results

In [ ]:
if results is not None:
    fig, axes = plt.subplots(3, 1, figsize=(14, 12))
    
    # Portfolio value
    axes[0].plot(results.index, results['portfolio_value'], linewidth=2)
    axes[0].set_title('Portfolio Value Over Time', fontsize=13, fontweight='bold')
    axes[0].set_ylabel('Value ($)', fontsize=11)
    axes[0].grid(True, alpha=0.3)
    axes[0].axhline(y=CAPITAL_BASE, color='r', linestyle='--', alpha=0.5, label='Starting Capital')
    axes[0].legend()
    
    # Cumulative returns
    cum_returns = (1 + results['returns']).cumprod() - 1
    axes[1].plot(results.index, cum_returns * 100, linewidth=2, color='green')
    axes[1].set_title('Cumulative Returns', fontsize=13, fontweight='bold')
    axes[1].set_ylabel('Return (%)', fontsize=11)
    axes[1].grid(True, alpha=0.3)
    axes[1].axhline(y=0, color='black', linestyle='-', alpha=0.3)
    
    # Number of positions
    axes[2].plot(results.index, results['num_positions'], linewidth=1.5, color='orange')
    axes[2].set_title('Number of Positions', fontsize=13, fontweight='bold')
    axes[2].set_ylabel('Positions', fontsize=11)
    axes[2].set_xlabel('Date', fontsize=11)
    axes[2].grid(True, alpha=0.3)
    axes[2].axhline(y=TOP_N_STOCKS, color='r', linestyle='--', alpha=0.5, label=f'Target ({TOP_N_STOCKS})')
    axes[2].legend()
    
    plt.tight_layout()
    plt.show()
    
    # Save results
    results_path = '/notebooks/multi_source_backtest_results.csv'
    results.to_csv(results_path)
    print(f"\n✓ Results saved to: {results_path}")
else:
    print("No results to visualize (backtest failed).")

## Summary

This notebook demonstrated:

### ✅ Multi-Source Data Integration
- Successfully loaded both Sharadar SF1 and custom LSEG fundamentals
- Used custom loader routing to access both sources simultaneously
- Compared metrics across sources for data quality validation

### ✅ Data Quality Analysis
- Compared ROE, P/E, D/E from both sources
- Identified overlapping coverage
- Calculated correlation and differences

### ✅ Consensus Scoring
- Built composite quality score using both sources
- Gave bonus points when LSEG confirms Sharadar signals
- Identified high-conviction picks where both sources agree

### ✅ Production-Ready Strategy
- Complete backtest using multi-source fundamentals
- Monthly rebalancing with quality screening
- Performance tracking and visualization

---

## Key Insights

**Coverage**:
- Sharadar provides broad coverage (~5,000+ stocks)
- Custom LSEG provides validation/confirmation
- Overlap depends on your LSEG universe

**Data Quality**:
- Compare metrics across sources to catch errors
- Use consensus when both sources available
- Fall back to Sharadar for broader coverage

**Strategy Design**:
- Primary screen: Sharadar (broader coverage)
- Validation: LSEG where available
- Confidence boost: Both sources agree

---

## Next Steps

1. **Expand Custom Universe**: Add more stocks to LSEG database for better coverage
2. **Add More Metrics**: Compare additional fundamental metrics
3. **Discrepancy Analysis**: Investigate large differences between sources
4. **Alpha Discovery**: Find stocks where sources disagree (potential mispricing)
5. **Production Deployment**: Use consensus scores in live trading

---

**Documentation**:
- Sharadar Guide: `docs/SHARADAR_FUNDAMENTALS_GUIDE.md`
- Custom Data Guide: `examples/custom_data/README.md`
- Database Class: `examples/custom_data/DATABASE_CLASS_GUIDE.md`